In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [55]:
def backtracking(f, grad_f, x, X, Y):
    """
    This function is a simple implementation of the backtracking algorithm for
    the GD (Gradient Descent) method.
    
    f: function. The function that we want to optimize.
    grad_f: function. The gradient of f(x).
    x: ndarray. The actual iterate x_k.
    """
    alpha = 1
    c = 0.8
    tau = 0.25
    
    while f(x - alpha * grad_f(x, X, Y), X, Y) > f(x, X, Y) - c * alpha * np.linalg.norm(grad_f(x, X, Y), 2) ** 2:
        alpha = tau * alpha
        
        if alpha < 1e-3:
            break
    return alpha

In [56]:
def gradient_descent(f, grad_f, x0, kmax, tolf, tolx, X, Y, do_backtracking,alpha_value=0.01):
    # Initialize lists to store the iterates, function values, gradients, and errors
    x = [x0]
    f_val = [f(x0, X, Y)]
    grads = [grad_f(x0, X, Y)]
    err = [np.linalg.norm(grad_f(x0, X, Y))]
    
    for k in range(kmax):
        # Compute next iterate we set the step size to 0.01
        if do_backtracking == True:
            alpha = backtracking(f, grad_f, x[-1], X, Y)
        else:
            alpha = alpha_value
        x_new = x[-1] - grad_f(x[-1], X, Y) * alpha

        # Update the lists
        x.append(x_new)
        f_val.append(f(x_new, X, Y))
        grads.append(grad_f(x_new, X, Y))
        err.append(np.linalg.norm(grad_f(x_new, X, Y)))
        
        # Check for convergence
        if np.linalg.norm(grad_f(x_new, X, Y)) < tolf * np.linalg.norm(grad_f(x0, X, Y)) or np.linalg.norm(x_new - x[-2]) < tolx:
            break

    return x, k, f_val, grads, err

In [57]:
def stochastic_gradient_descent(l, grad_l, w0, data, batch_size, n_epochs,alpha):
    # Initialize lists to store the iterates, function values, gradients, and errors
    w = [w0]
    f_val = []
    grads = []
    err = []

    # Unpack the data
    x, y = data

    # Compute the number of batches
    n_batches = len(y) // batch_size

    for epoch in range(n_epochs):
        indices = np.arange(len(y))
        # Shuffle the indices
        np.random.shuffle(indices)


        # Shuffle the data
        #indices = np.random.permutation(len(y))
        x = x[:,indices]
        y = y[indices]

        for i in range(n_batches):
            # Select the data for the current batch
            x_batch = x[:,i * batch_size:(i + 1) * batch_size]
            y_batch = y[i * batch_size:(i + 1) * batch_size]

            # Compute the gradient for the current batch
            grad = grad_l(w[-1], x_batch, y_batch)

            # Update the iterate
            w_new = w[-1] - alpha* grad
            w.append(w_new)

        # Compute the function value, gradient, and error after each epoch
        f_val.append(l(w[-1], x, y))
        grad_epoch = grad_l(w[-1], x, y)
        grads.append(grad_epoch)
        err.append(np.linalg.norm(grad_epoch))

    return w, f_val, grads, err

#theta_list_sgd, f_vals_sgd, grad_vals_sgd, error_vals_sgd = stochastic_gradient_descent_mle(f_mle, grad_f_mle, np.random.randn(K), (Phi_X_train, Y_train), 64, 10000,0.01)

In [58]:
#import data
data = pd.read_csv('data.csv')

# Split the dataset into X and Y
Xt = (data.drop('label', axis=1).values).T
Yt = (data['label'].values).T
print(Xt.shape)

(784, 42000)


In [59]:
def split_data(X, Y, Ntrain):
    d, N = X.shape

    idx = np.arange(N)
    np.random.shuffle(idx)

    train_idx = idx[:Ntrain]
    test_idx = idx[Ntrain:]

    Xtrain = X[:, train_idx]
    Ytrain = Y[train_idx]
    
    Xtest = X[:, test_idx]
    Ytest = Y[test_idx]

    return (Xtrain, Ytrain), (Xtest, Ytest)


print(X_1.shape)

(784, 8423)


In [60]:
# Define sigmoid
def sigmoid(X):
    return 1 / (1 + np.exp(-X))


def f(xhat,w):
    return sigmoid(xhat.T @ w)

# Value of the loss
def ell(w, X, Y):
    N = len(X)
    return (1/(2*N)) * np.linalg.norm(f(X, w) - Y,2)**2

#Sum of MSE
#def ell(w, X, Y):
    #return np.mean((Y - f(X, w))**2)

# Value of the gradient
def grad_ell(w, X, Y):
    # print(w.shape)
    # print(X.shape)
    # print(Y.shape)
    temp = f(X,w)
    # print(temp.shape)
    return (1/len(Y)) * X @ (f(X, w) * (1 - f(X, w)) * (f(X, w) - Y))




In [81]:

def logistic_regression(input1,input2,X, Y, ell, grad_ell, kmax, tolf, tolx, do_backtracking,alpha_value=0.01, SGD_or_GD=1, epoch=3, batch_size=10,train_ratio=0.8):
   #filter the data to only include the inputs the user entered
    mask = (Y == int(input1)) | (Y == int(input2))

    # Use the mask to extract the corresponding columns from X
    X_1 = X[:, mask]
    Y_1 = Y[mask]
    Y_1[Y_1 == int(input1)] = 0
    Y_1[Y_1 == int(input2)] = 1
   
    # Split the data into training and testing sets
    (Xtrain, Ytrain), (Xtest, Ytest) = split_data(X_1, Y_1, int(train_ratio*len(Y_1)))

    np.random.seed(0)
    w0= np.random.normal(0, 0.01, Xtrain.shape[0])
    if SGD_or_GD == 1:
        w, k, f_val, grads, err= gradient_descent(ell, grad_ell, w0, kmax, tolf, tolx,Xtrain,Ytrain, do_backtracking,alpha_value)
    else:
        w, f_val, grads, err=stochastic_gradient_descent(ell, grad_ell, w0, (Xtrain, Ytrain), batch_size, epoch,alpha_value)
    
    last_w = w[-1]
    # print(last_w)

    correct = 0
    for i in range(len(Ytest)):
        # print(f(Xtest[:,i], last_w), Ytest[i])
        if f(Xtest[:,i], last_w)>0.5:
            if Ytest[i] == 1:
                correct += 1
        else:
            if Ytest[i] == 0:
                correct += 1
        
        # print(f(Xtest[:,i], last_w), Ytest[i])
    
    return correct/len(Ytest), last_w

acc = logistic_regression(5,3,Xt,Yt, ell, grad_ell,100, 1e-6, 1e-6, True, 0.01, 0, 1, 64,0.5)[0]

train_ratios = [0.1,0.8]
inputs=np.array([(5,3),(3,4),(1,7)])
    
for input1, input2 in inputs:
    print("Inputs", input1, input2)
    for train_ratio in train_ratios:
        acc_gd, w_gd = logistic_regression(input1,input2,Xt,Yt, ell, grad_ell,100, 1e-6, 1e-6, False, 0.01, 1, 3, 64,train_ratio)
        acc_sgd, w_sgd = logistic_regression(input1,input2,Xt,Yt, ell, grad_ell,100, 1e-6, 1e-6, False, 0.01, 0, 3, 64,train_ratio)
        print("Accuracy for GD with train ratio", train_ratio, "is", acc_gd)
        print("Accuracy for SGD with train ratio", train_ratio, "is", acc_sgd)
        print("Difference of weights between GD and SGD is", np.linalg.norm(w_gd - w_sgd),"\n")

    print("\n")

Inputs 5 3
Accuracy for GD with train ratio 0.1 is 0.466721222040371
Accuracy for SGD with train ratio 0.1 is 0.4660392798690671
Difference of weights between GD and SGD is 0.05872645249412016 

Accuracy for GD with train ratio 0.8 is 0.454601226993865
Accuracy for SGD with train ratio 0.8 is 0.9
Difference of weights between GD and SGD is 0.22763479421920524 



Inputs 3 4
Accuracy for GD with train ratio 0.1 is 0.9891834850283604
Accuracy for SGD with train ratio 0.1 is 0.9858857670492019
Difference of weights between GD and SGD is 0.11930496526216118 

Accuracy for GD with train ratio 0.8 is 0.9905044510385757
Accuracy for SGD with train ratio 0.8 is 0.9910979228486647
Difference of weights between GD and SGD is 0.19977860665447045 



Inputs 1 7
Accuracy for GD with train ratio 0.1 is 0.48220618808854104
Accuracy for SGD with train ratio 0.1 is 0.5169377522318699
Difference of weights between GD and SGD is 0.15512614598126362 

Accuracy for GD with train ratio 0.8 is 0.986791414419